## BERT의 문장 임베딩(SBERT)을 이용한 한국어 챗봇

SBERT를 이용하여 문장 임베딩을 얻을 수 있는 패키지인 sentence_transformers를 사용하여 쉽고 간단하게 한국어 챗봇을 구현해봅시다. 실습에 앞서 sentence_transformers를 설치합니다.
```
pip install sentence_transformers
```
트랜스포머를 이용한 한국어 챗봇 실습에서 사용했던 데이터를 그대로 사용합니다.

In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

2022-04-12 18:18:32.704473: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-12 18:18:32.704508: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
train_data = pd.read_csv('data/catechism/catechism.csv')
train_data.head()

,No,Q,A
0,1,사람의 제일 되는 목적이 무엇인가?,사람의 제일 되는 목적은 하나님을 영화롭게 하는 것과 영원토록 그를 즐거워하는 것이...
1,2,하나님께서 무슨 규칙을 우리에게 주시어 어떻게 자기를 영화롭게 하고 즐거워할 것을 ...,신구약 성경에 기재된 하나님의 말씀은 어떻게 우리가 그를 영화롭게 하고 즐거워할 것...
2,3,성경이 제일 요긴하게 교훈하는 것이 무엇인가?,성경이 제일 요긴하게 교훈하는 것은 사람이 하나님에 대하여 어떻게 믿을 것과 하나님...
3,4,하나님은 어떤 분이신가?,하나님은 신이신데 그의 존재하심과 지혜와 권능과 거룩하심과 공의와 인자하심과 진실하...
4,5,하나님 한분밖에 또 다른 하나님이 계신가?,"한분뿐이시니 참되시며 살아 계신 하나님이시다.(신 6:4, 렘 10:10, 요 17..."


문장 임베딩을 얻기 위해서 사전 훈련된 BERT를 로드합니다. 여기서는 한국어도 포함되어 학습된 다국어 모델을 로드합니다.
  
모델의 이름은 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens'인데 이름이 의미하는 바는 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다는 의미입니다. 다시 말해서 NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미입니다.

In [3]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

SentenceTransformer로 로드할 수 있는 다양한 모델에 대한 리스트는 아래의 링크에서 확인 가능합니다.
해당 링크에 한국어 버전의 모델들 또한 공개되어 있으니 방문해보세요.

링크 : https://huggingface.co/models?library=sentence-transformers

데이터에서 모든 질문열. 즉, train_data['Q']에 대해서 문장 임베딩 값을 구한 후 embedding이라는 새로운 열에 저장합니다.

In [4]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [5]:
train_data[:5]

,No,Q,A,embedding
0,1,사람의 제일 되는 목적이 무엇인가?,사람의 제일 되는 목적은 하나님을 영화롭게 하는 것과 영원토록 그를 즐거워하는 것이...,"[0.066811085, 0.15771681, 1.0360198, 0.5023864..."
1,2,하나님께서 무슨 규칙을 우리에게 주시어 어떻게 자기를 영화롭게 하고 즐거워할 것을 ...,신구약 성경에 기재된 하나님의 말씀은 어떻게 우리가 그를 영화롭게 하고 즐거워할 것...,"[0.18245241, 0.590516, 1.0600048, -0.22893694,..."
2,3,성경이 제일 요긴하게 교훈하는 것이 무엇인가?,성경이 제일 요긴하게 교훈하는 것은 사람이 하나님에 대하여 어떻게 믿을 것과 하나님...,"[0.24954857, 0.1462625, 0.71617454, -0.1964443..."
3,4,하나님은 어떤 분이신가?,하나님은 신이신데 그의 존재하심과 지혜와 권능과 거룩하심과 공의와 인자하심과 진실하...,"[0.23546648, 0.7400266, 1.0294378, -0.17563245..."
4,5,하나님 한분밖에 또 다른 하나님이 계신가?,"한분뿐이시니 참되시며 살아 계신 하나님이시다.(신 6:4, 렘 10:10, 요 17...","[0.32943794, 0.30873445, 0.5993381, -0.1840098..."


두 개의 벡터로부터 코사인 유사도를 구하는 함수 cos_sim를 정의합니다.

In [6]:
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

return_answer 함수는 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열. 즉, train_data['embedding']에 저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 코사인 유사도 값이 가장 높은 질문 샘플을 찾아냅니다. 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴합니다.

In [7]:
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

이제 챗봇을 테스트해봅시다.

In [12]:
return_answer('하나님은 어떤 분')

'하나님은 신이신데 그의 존재하심과 지혜와 권능과 거룩하심과 공의와 인자하심과 진실하심이 무한하시며, 무궁하시며, 불변하시다.(요 4:24, 시 90:2, 말 3:6, 약 1:17, 왕상 8:27, 렘 23:24, 사 40:22, 시 147:5, 롬 16:27, 창 17:1, 계 19:6, 사 57:15, 요 17:11, 계 4:8, 신 32:4, 시 100:5, 롬 2:4, 출 34:6, 시 117:2, 출 3:14, 시 145:3)'

In [22]:
return_answer('하나님을 믿는것')

'하나님의 신격에 삼위가 계시니 성부와 성자와 성령이신데 이 삼위는 한 하나님이시다. 본체는 하나요 권능과 영광은 동등이시다.(고후 13:13, 마 3:16∼17, 28:19, 고후 8:14, 요 1:1, 4:18, 행 5:3∼4, 히 1:3)'

In [18]:
return_answer('성경의 교훈')

'성경이 제일 요긴하게 교훈하는 것은 사람이 하나님에 대하여 어떻게 믿을 것과 하나님께서 사람에게 요구하시는 본분이다.(미 6:8, 요 5: 39, 20:31, 3:16, 고전 10:11, 롬 15:4, 요일 1:3∼4)'